<a href="https://colab.research.google.com/github/jx-dohwan/Emotion_recognition_in_conversation.ipynb/blob/main/Emotion_recognition_in_conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.25.1
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
torch.__version__

'1.13.1+cu116'

In [3]:
import transformers
transformers.__version__

'4.25.1'

In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



##  데이터 다운로드

In [5]:
!git clone https://github.com/declare-lab/MELD.git

fatal: destination path 'MELD' already exists and is not an empty directory.


In [6]:
import glob # 어떤 리스트가 있는지 도출하는 것
data_path = "./MELD/data/MELD/*csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/test_sent_emo.csv', './MELD/data/MELD/dev_sent_emo.csv', './MELD/data/MELD/train_sent_emo.csv']


## 데이터 확인

In [7]:
!head -5 './MELD/data/MELD/dev_sent_emo.csv' # head는 해당 파일의 앞부분을 보겠다.

Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
1,"Oh my God, he’s lost it. He’s totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
3,"Or! Or, we could go to the bank, close our accounts and cut them off at the source.",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
4,You’re a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"


In [8]:
# 데이터 출력
import csv #csv 확장자를 읽어들임임
for data_path in data_path_list:
  f = open(data_path, 'r')
  rdr = csv.reader(f)

  for line in rdr:
    print(line)
    break

  f.close()
  break


['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime']


## 세션으로 데이터 분할하기

[발화1, 발화2, 발화3] --> 감정예측한다는 것은 발화3에 해당하는 감정을
여기서 발화1, 발화2에 해당하는 감정이 있고, 이를 학습데이터로 사용해야된다.
--> [발화1], [발화1, 발화2]

//
모델이 roberta: bidirectional
: attention이 양방향

[발화1, 발화2, 발화3]: 여기서 발화2에 해당하는 감정을 학습한다.
: 동시에 발화1에 해당하는 감정을 학습한다.

In [9]:
# 데이터 저장
# roberta : bidirectional 
# attention이 양방향
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])    
    return final_data
    
for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)# 리스트 하나씩 데이터 불러읽어 들이기
    
    """ 세션 데이터 저장할 것"""
    session_dataset = []
    session = []
    speaker_set = []
    
    """ 실제 데이터 저장 방식 """
    pre_sess = 'start'
    for i, line in enumerate(rdr):
        if i == 0:
            """ 저장할 데이터들 index 확인 """
            header  = line
            utt_idx = header.index('Utterance')
            speaker_idx = header.index('Speaker')
            emo_idx = header.index('Emotion')
            sess_idx = header.index('Dialogue_ID')
        else:
            utt = line[utt_idx]
            speaker = line[speaker_idx]
            """ 유니크한 스피커로 바꾸기 """
            if speaker in speaker_set:# 스키퍼셋안에 있는 스피커(즉 이미 있는 스피커라면)
                uniq_speaker = speaker_set.index(speaker)# 어떤 사람이 말했는지 그 사람의 index를 uniq로 둔다.
            else:# 처음 등장한 인물이라면
                speaker_set.append(speaker)# 스피커셋에 스피커를 넣어주고
                uniq_speaker = speaker_set.index(speaker)#유니크 스피커가 몇 번째 인덱스에 해당하는지 바꿔줌
            emotion = line[emo_idx]
            sess = line[sess_idx]
            
            if pre_sess == 'start' or sess == pre_sess:# sess이 pre_sess이 같거나 start일때 저장
                session.append([uniq_speaker, utt, emotion])
            else:#다를경우 초기화하고 다시저장?
                """ 세션 데이터 저장 """
                # session_dataset.append(session)
                session_dataset += split(session)# 서브세셔으로 분리하여 데이터로 바꿈
                session = [[uniq_speaker, utt, emotion]]
                speaker_set = []
            pre_sess = sess   
    """ 마지막 세션 저장 """
    # session_dataset.append(session)
    session_dataset += split(session)
    f.close()
    
    """ 데이터 분할하기 """
    break

In [10]:
session_dataset[0]

[[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise']]

In [11]:
session_dataset[1]

[[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise'],
 [1,
  'Oh. That’s so Monica can keep track. That way if one on them is missing, she can be like, ‘Where’s number 27?!’',
  'anger']]

## 실제 감정인식에 맞게 데이터 분할하기(배치처리 알아보기)

In [12]:
import csv
from torch.utils.data import Dataset
def split(session):
    final_data = []
    split_session =[]
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)

        """추가"""
        emoSet = set()

        """ 세션 데이터 저장할 것"""
        self.session_dataset = []
        session = []
        speaker_set = []

        """ 실제 데이터 저장 방식 """
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                """ 저장할 데이터들 index 확인 """
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]
                """ 유니크한 스피커로 바꾸기 """
                if speaker in speaker_set:# 스키퍼셋안에 있는 스피커(즉 이미 있는 스피커라면)
                    uniq_speaker = speaker_set.index(speaker)# 어떤 사람이 말했는지 그 사람의 index를 uniq로 둔다.
                else:# 처음 등장한 인물이라면
                    speaker_set.append(speaker)# 스피커셋에 스피커를 넣어주고
                    uniq_speaker = speaker_set.index(speaker)#유니크 스피커가 몇 번째 인덱스에 해당하는지 바꿔줌
                emotion = line[emo_idx]
                sess = line[sess_idx]
                
                if pre_sess == 'start' or sess == pre_sess:# sess이 pre_sess이 같거나 start일때 저장
                    session.append([uniq_speaker, utt, emotion])
                else:#다를경우 초기화하고 다시저장?
                    """ 세션 데이터 저장 """
                    # session_dataset.append(session)
                    self.session_dataset += split(session)# 서브세셔으로 분리하여 데이터로 바꿈
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess   
        """ 마지막 세션 저장 """
        # session_dataset.append(session)
        self.session_dataset += split(session)

        self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        # self.emoList = ['anger', 'disgust', 'fear',' joy', 'neutral', 'sadness', 'surprise']
        f.close()
    
    def __len__(self): # Dataset 기본적인 구성
        return len(self.session_dataset) ## 리스트의 길이이

    def __getitem__(self, idx): #기본적인 구성
        return self.session_dataset[idx] ## 리스트중 하나를 불러와서 사용함

    def collate_fn(self, sessions): # 배치를 위한 구성, sessions는 session_dataset모두 들어옴
        '''
            input:
                data : [(session1),(session2),...]
            return:
                batch_input_tokens_pad: (B, L) apdded
                batch_labels: (B)
        '''
        batch_input_token = []
        for session in sessions:
            input_token = ""
            for line in session: # 하나의 세션을 linebyline으로 쪼개서 input_token으로 만든다.
                speaker, utt, emotion = line
                input_token += utt
            batch_input_token.append(input_token)
        return batch_input_token      

In [13]:
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataset[0], dev_dataset[1]

([[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness']],
 [[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness'],
  [1, 'What?', 'surprise']])

In [14]:
"""배치결과 확인 """
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, 
                            batch_size=3, 
                            shuffle=False,
                            num_workers=4, # 메모리를 몇 개를 사용할지?
                            collate_fn=dev_dataset.collate_fn)

i = 0
for data in dev_dataloader:
    print(i, data)
    i += 1
    if i > 2:
        break

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0 ['Oh my God, he’s lost it. He’s totally lost it.', 'Oh my God, he’s lost it. He’s totally lost it.What?', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.']
1 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.']
2 ['Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.Hey.', 'Or! Or, we could go to the bank, close our accounts and cut them off at the source.You’re a genius!Aww, man, now we won’t be bank buddies!Now, there’s two reasons.Hey.Hey!', 'Or! Or, we could go to the bank, close o

## 사전학습 모델들에 대한 백그라운드


In [15]:
""" 토크나이즈 확인하기"""
# https://github.com/thunlp/PLMpapers
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [16]:
print(tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id)

<s> </s> <pad>
0 2 1


In [17]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_create_trie',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager

In [18]:
tokenizer.model_max_length

512

In [19]:
""" 토크나이저 작동 """
res = tokenizer('hello. this is fastcampus')
print(res)
res = tokenizer.encode('hello. this is fastcampus')
print(res)
res = tokenizer(['hello. this is fastcampus','what are you doing'])
print(res)
res = tokenizer(['hello. this is fastcampus','what are you doing'], add_special_tokens=False)
print(res)
res = tokenizer.encode('hello. this is fastcampus', add_special_tokens=False)
print(res)

{'input_ids': [0, 42891, 4, 42, 16, 1769, 28135, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[0, 42891, 4, 42, 16, 1769, 28135, 2]
{'input_ids': [[0, 42891, 4, 42, 16, 1769, 28135, 2], [0, 12196, 32, 47, 608, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
{'input_ids': [[42891, 4, 42, 16, 1769, 28135], [12196, 32, 47, 608]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}
[42891, 4, 42, 16, 1769, 28135]


In [22]:
import csv
from torch.utils.data import Dataset
def split(session):
    final_data = []
    split_session =[]
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)

        """추가"""
        emoSet = set()
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

        """ 세션 데이터 저장할 것"""
        self.session_dataset = []
        session = []
        speaker_set = []

        """ 실제 데이터 저장 방식 """
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                """ 저장할 데이터들 index 확인 """
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]
                """ 유니크한 스피커로 바꾸기 """
                if speaker in speaker_set:# 스키퍼셋안에 있는 스피커(즉 이미 있는 스피커라면)
                    uniq_speaker = speaker_set.index(speaker)# 어떤 사람이 말했는지 그 사람의 index를 uniq로 둔다.
                else:# 처음 등장한 인물이라면
                    speaker_set.append(speaker)# 스피커셋에 스피커를 넣어주고
                    uniq_speaker = speaker_set.index(speaker)#유니크 스피커가 몇 번째 인덱스에 해당하는지 바꿔줌
                emotion = line[emo_idx]
                sess = line[sess_idx]
                
                if pre_sess == 'start' or sess == pre_sess:# sess이 pre_sess이 같거나 start일때 저장
                    session.append([uniq_speaker, utt, emotion])
                else:#다를경우 초기화하고 다시저장?
                    """ 세션 데이터 저장 """
                    # session_dataset.append(session)
                    self.session_dataset += split(session)# 서브세셔으로 분리하여 데이터로 바꿈
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess   
        """ 마지막 세션 저장 """
        # session_dataset.append(session)
        self.session_dataset += split(session)

        #self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        f.close()
    
    def __len__(self): # Dataset 기본적인 구성
        return len(self.session_dataset) ## 리스트의 길이이

    def __getitem__(self, idx): #기본적인 구성
        return self.session_dataset[idx] ## 리스트중 하나를 불러와서 사용함

    def padding(self, batch_input_token):# batch가 1이면 필요없다./ 길이가 다른 입력을 처리할 필요가없기 때문에
        """추가"""
        """ 512 토큰 길이 넘으면 잘라내기"""
        batch_token_ids, batch_attention_masks = batch_input_token['input_ids'], batch_input_token['attention_mask'] # 
        trunc_batch_token_ids, trunc_batch_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(batch_token_ids, batch_attention_masks):
            if len(batch_token_id) > self.tokenizer.model_max_length: # 512보다 길이가 긴경우우
                trunc_batch_token_id = [batch_token_id[0]] + batch_token_id[1:][-self.tokenizer.model_max_length+1:] 
                # cls는 필수, 1부터 길이만큼인데 뒤부분부터 자름 앞부분부터 자르면 마지막 발화에 감정 예측 토큰이 사라지면 학습이 제대로 되지 않는다. 
                trunc_batch_attention_mask = [batch_attention_mask[0]] + batch_attention_mask[1:][-self.tokenizer.model_max_length+1:]
                # attention mask도 위와 동일하게
                trunc_batch_token_ids.append(trunc_batch_token_id)
                trunc_batch_attention_masks.append(trunc_batch_attention_mask)
            else: # 512보다 길이가 짧은 경우 -> 그냥 들어간다.
                trunc_batch_token_ids.append(batch_token_id)
                trunc_batch_attention_masks.append(batch_attention_mask)
        """ padding token으로 패딩하기"""
        #
        max_length = max([len(x) for x in trunc_batch_token_ids])
        padding_tokens, padding_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(trunc_batch_token_ids, trunc_batch_attention_masks):
            padding_tokens.append(batch_token_id + [self.tokenizer.pad_token_id for _ in range(max_length-len(batch_token_id))])
            # 512-max_length해서 남는 부분을 padding 토큰 입력
            padding_attention_masks.append(batch_attention_mask + [0 for _ in range(max_length-len(batch_token_id))])
            # 0을 넣어준다.
        return torch.tensor(padding_tokens), torch.tensor(padding_attention_masks) # 리스트가 텐서포 변함

    def collate_fn(self, sessions): # 배치를 위한 구성, sessions는 session_dataset모두 들어옴
        '''
            input:
                data : [(session1),(session2),...]
            return:
                batch_input_tokens_pad: (B, L) apdded
                batch_labels: (B)
        '''
        # 컨텍스트 길이 조정해도 된다.
        # 발화 1이런식으로 앞에 제거할 수 있다. 
        # 앞을 제구하는 이유는 뒤에 내용이 더 중요하기 때문이다.
        """추가"""
        batch_input, batch_labels = [], [] # com 입력, 학습할 레이블
        batch_PM_input = [] # pm 입력
        for session in sessions: # 하나의 세션을 linebyline으로 쪼개서 input_token으로 만든다.
            input_str = self.tokenizer.cls_token # 입력의 맨 앞에 들어감
            """For PM"""
            current_speaker, curent_utt, current_emotion = session[-1] # 마지막을 의미함/마지막 발화자를 알기 위함
            PM_input = [] 
            for i, line in enumerate(session): # 세션에서 하나씩 부르는데 
                speaker, utt, emotion = line
                input_str += " " + utt + self.tokenizer.sep_token # 중첩해서 더해줌 구분은 sep로 
                if i < len(session)-1 and current_speaker == speaker: # 마지막은 무조건 스피커와 커런트가 같으니까 아닌 경우에 대해 입력해줌/스피커와 커런트 스피커와 같으면 입력
                    PM_input.append(self.tokenizer.encode(utt, add_special_tokens=True, return_tensors='pt')) 
                    # True : concat해서 저장하는 것이 아니고 하나의 입력을 바로 PM에 태우는 것이기 때문
                    # [cls_token, tokens, sep_token]
            
            """For CoM"""
            batch_input.append(input_str)
            batch_labels.append(self.emoList.index(emotion))
            batch_PM_input.append(PM_input) # batch안에 batch의 개념
        batch_input_token = self.tokenizer(batch_input, add_special_tokens=False)
        batch_padding_token, batch_padding_attention_mask = self.padding(batch_input_token)

        return batch_padding_token, batch_padding_attention_mask, batch_PM_input, torch.tensor(batch_labels)      

In [23]:
""" 배치 결과 확인 """
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

torch.Size([3, 58])
batch_padding_token tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
            17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,  

## 파일로 저장하기

In [24]:
import csv
from torch.utils.data import Dataset
from transformers import RobertaTokenizer
import torch

In [25]:
RobertaTokenizer

transformers.models.roberta.tokenization_roberta.RobertaTokenizer

In [26]:
!touch dataset.py
# 코드 복사하기

In [27]:
import sys
sys.path.append('/content/')
from dataset import data_loader
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

torch.Size([3, 58])
batch_padding_token tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
            17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,  

## 사전학습 모델 불러와 로딩하기

In [28]:
from transformers import RobertaModel
model = RobertaModel.from_pretrained('roberta-base')
print('')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
model 

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [30]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_can_retrieve_inputs_from_name',
 '_convert_head_mask_to_5d',
 '_create_repo',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_from_config',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_files_timestamps',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_resized_embeddings',
 '_get_resized_lm_head',
 '_get_stopp

In [31]:
#model.train() # dropout의 probability을 이용한다.
#model.eval() # dropout의 probability을 이용용한다. -> dropout 작동하지 않는다.

## 사전학습 모델 사용해보기
- 이 부분 너무 어렵다.

In [32]:
batch_padding_token.shape, batch_padding_attention_mask.shape

(torch.Size([3, 58]), torch.Size([3, 58]))

In [33]:
batch_padding_token, batch_padding_attention_mask

(tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
             17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213, 

In [34]:
""" for CoM """
batch_com_out = model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
print(batch_com_out.shape)
batch_com_final = batch_com_out[:,0,:] # CLS 토큰의 output 가져오기 위해
print(batch_com_final.shape)

torch.Size([3, 58, 768])
torch.Size([3, 768])


In [35]:
result = model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)
result.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [36]:
import torch

model2 = RobertaModel.from_pretrained('roberta-base')
# 발화1: feature1 [1, 768]
# 발화3: feature3 [1, 768]
# 발화6에 해당하는 감정을 예측할 때 발화1, 발화3의 정보를 사용할 것
# feature1 + feature3
# (feature1, feature6) 어텐션 weights w1
# (feature3, feature6) 어텐션 weights w3
# w1*feature1 + w3*feature6
# GRU(feature1, feature3)

""" GRU 세팅 """
import torch.nn as nn 
hiddenDim = model2.config.hidden_size
zero = torch.empty(2, 1, hiddenDim)
h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
speakerGRU = nn.GRU(hiddenDim, hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)

""" GRU 통과 --> PM 결과 """
batch_pm_gru_final = []
for PM_inputs in batch_PM_input:
    if PM_inputs:
        pm_outs = []
        for PM_input in PM_inputs:
            pm_out = model2(PM_input)['last_hidden_state'][:,0,:] # CLS의 출력/attention에 해당하는 것을 명시하지 않고 토큰들만 넣어 pm 출력 뽑아냄 그중 CLS
            pm_outs.append(pm_out)
        pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)로 만듬 토치텐서
        pm_gru_outs, _ = speakerGRU(pm_outs, h0) # (speaker_num, batch=1, hidden_dim)로 만듬 토치텐서서
        pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim) 마지막것이 중요하니 해당하는 것을 가져와서 사용함
        batch_pm_gru_final.append(pm_gru_final)
    else:
        batch_pm_gru_final.append(torch.zeros(1, hiddenDim)) #pm입력이 없는 경우 torch zero를 넣어준다.
batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
batch_pm_gru_final.shape

torch.Size([3, 768])

## 추가적인 layer 구성하기

In [38]:
dev_dataset.emoList

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [39]:
""" score matrix """
clsNum = len(dev_dataset.emoList)
W = nn.Linear(hiddenDim, clsNum)
final_output = W(batch_com_final + batch_pm_gru_final)
print(final_output.shape) # (B, C)

torch.Size([3, 7])


In [40]:
from transformers import RobertaModel
import torch
import torch.nn as nn

class ERC_model(nn.Module):
    def __init__(self, clsNum):
        super(ERC_model, self).__init__()
        self.com_model = RobertaModel.from_pretrained('roberta-base')
        self.pm_model = RobertaModel.from_pretrained('roberta-base')
        
        """ GRU 세팅 """
        self.hiddenDim = self.com_model.config.hidden_size
        zero = torch.empty(2, 1, self.hiddenDim)
        self.h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
        self.speakerGRU = nn.GRU(self.hiddenDim, self.hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)
        
        """ score matrix """
        self.W = nn.Linear(self.hiddenDim, clsNum)
    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        """ for CoM """
        batch_com_out = self.com_model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
        batch_com_final = batch_com_out[:,0,:]
        
        """ GRU 통과 --> PM 결과 """
        batch_pm_gru_final = []
        for PM_inputs in batch_PM_input:
            if PM_inputs:
                pm_outs = []
                for PM_input in PM_inputs:
                    pm_out = self.pm_model(PM_input)['last_hidden_state'][:,0,:]
                    pm_outs.append(pm_out)
                pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
                pm_gru_outs, _ = self.speakerGRU(pm_outs, self.h0) # (speaker_num, batch=1, hidden_dim)
                pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
                batch_pm_gru_final.append(pm_gru_final)
            else:
                batch_pm_gru_final.append(torch.zeros(1, self.hiddenDim))
        batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)        
        
        """ score matrix """
        final_output = self.W(batch_com_final + batch_pm_gru_final) # (B, C)
        
        return final_output

In [41]:
clsNum = len(dev_dataset.emoList)
model = ERC_model(clsNum)
result = model(batch_padding_token, batch_padding_attention_mask, batch_PM_input) # forward에 들어가서 동작작
print(result.shape)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaMod

torch.Size([3, 7])


## 파일로 저장

In [ ]:
!touch model.py
# 코드 복사하기